In [1]:
# Import libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import keras
import tensorflow as tf
import warnings
import os
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
warnings.filterwarnings('ignore')

In [2]:
url = 'Datasets/2012_2019_PlatteRiverWeir_features_merged_all.csv'

df = pd.read_csv(url)

In [3]:
df

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WeirPt2X,WeirPt2Y,WwRawLineMin,WwRawLineMax,WwRawLineMean,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma
0,2012-06-09 13:15:00,2012-06-09T13:09:07,StateLineWeir_20120609_Farrell_001.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:28,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,2012-06-09 13:15:00,2012-06-09T13:10:29,StateLineWeir_20120609_Farrell_002.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:33,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,2012-06-09 13:45:00,2012-06-09T13:44:01,StateLineWeir_20120609_Farrell_003.jpg,USGS,6674500,MDT,2.96,873.0,2020-03-11T16:58:40,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,2012-06-09 14:45:00,2012-06-09T14:44:30,StateLineWeir_20120609_Farrell_004.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:47,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,2012-06-09 15:45:00,2012-06-09T15:44:59,StateLineWeir_20120609_Farrell_005.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:55,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42054,2019-10-11 09:00:00,2019-10-11T08:59:53,StateLineWeir_20191011_Farrell_409.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:53,4288,...,2446,1900,9284.0,77521.0,38385.370066,15952.029728,0.0,70085.0,37550.894823,16444.401209
42055,2019-10-11 10:00:00,2019-10-11T09:59:52,StateLineWeir_20191011_Farrell_410.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:54,4288,...,2440,1900,10092.0,74614.0,40162.989292,15467.708856,0.0,70061.0,39397.339095,16009.008049
42056,2019-10-11 11:00:00,2019-10-11T10:59:52,StateLineWeir_20191011_Farrell_411.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:55,4288,...,2447,1900,7067.0,83260.0,42095.946590,16770.357949,0.0,76335.0,41350.006568,17489.374617
42057,2019-10-11 12:00:00,2019-10-11T11:59:53,StateLineWeir_20191011_Farrell_412.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:56,4288,...,2443,1900,6283.0,83045.0,45345.490954,17498.432849,0.0,78882.0,44553.920296,18268.294896


In [9]:
imglist = []
for path in os.listdir('Datasets/images/')+df[' Filename']:
    imglist.append(os.path.join('Datasets/images/', path))

In [10]:
df2 = df
df2['images'] = imglist
df2 = df2.sample(frac=1).reset_index(drop=True)
df2

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WeirPt2Y,WwRawLineMin,WwRawLineMax,WwRawLineMean,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma,images
0,2015-04-18 09:00:00,2015-04-18T08:57:44,StateLineWeir_20150418_Farrell_975.JPG,USGS,6674500,MDT,2.24,193.0,2020-03-11T23:03:07,4288,...,1900,1832.0,67896.0,29953.746455,16362.030404,0.0,61859.0,29653.600833,16306.845477,Datasets/images/StateLineWeir_20150418_Farrell...
1,2017-09-02 14:00:00,2017-09-02T13:59:49,StateLineWeir_20170902_Farrell_067.jpg,USGS,6674500,MDT,3.56,1630.0,2020-03-12T08:31:37,4288,...,1900,3291.0,120446.0,90582.245797,28473.425686,0.0,118124.0,89406.285600,30296.151189,Datasets/images/StateLineWeir_20170902_Farrell...
2,2018-04-07 17:15:00,2018-04-07T17:14:34,StateLineWeir_20180407_Farrell_339.jpg,USGS,6674500,MDT,2.29,224.0,2020-03-12T10:24:48,4288,...,1900,1550.0,73244.0,29376.797707,18192.102384,0.0,64460.0,29188.783142,17965.580142,Datasets/images/StateLineWeir_20180407_Farrell...
3,2017-06-06 11:00:00,2017-06-06T11:00:17,StateLineWeir_20170606_Farrell_642.jpg,USGS,6674500,MDT,4.58,3320.0,2020-03-12T07:08:01,4288,...,1900,0.0,97160.0,41404.200840,24985.793050,0.0,91560.0,41255.654072,24587.156055,Datasets/images/StateLineWeir_20170606_Farrell...
4,2017-06-18 05:00:00,2017-06-18T05:00:15,StateLineWeir_20170618_Farrell_966.jpg,USGS,6674500,MDT,3.85,2080.0,2020-03-12T07:04:29,4288,...,1900,0.0,75128.0,36370.513536,20389.465149,0.0,69835.0,36189.672951,20127.291125,Datasets/images/StateLineWeir_20170618_Farrell...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42054,2017-04-11 05:15:00,2017-04-11T05:15:09,StateLineWeir_20170411_Farrell_070.jpg,USGS,6674500,MDT,2.79,579.0,2020-03-12T06:54:25,4288,...,1900,0.0,81822.0,35027.765523,20934.734921,0.0,76676.0,34905.093878,20211.447286,Datasets/images/StateLineWeir_20170411_Farrell...
42055,2017-04-23 09:00:00,2017-04-23T09:00:12,StateLineWeir_20170423_Farrell_413.jpg,USGS,6674500,MDT,4.24,2720.0,2020-03-12T06:45:03,4288,...,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,Datasets/images/StateLineWeir_20170423_Farrell...
42056,2019-02-16 08:00:00,2019-02-16T07:59:20,StateLineWeir_20190216_Farrell_730.jpg,USGS,6674500,MST,2.17,164.0,2020-03-12T13:26:26,4288,...,1900,374.0,35985.0,16275.950123,10373.985930,0.0,33548.0,16201.727124,10127.155165,Datasets/images/StateLineWeir_20190216_Farrell...
42057,2014-07-27 10:00:00,2014-07-27T10:00:18,StateLineWeir_20140727_Farrell_934.JPG,USGS,6674500,MDT,3.25,1470.0,2020-03-11T21:33:57,4288,...,1900,0.0,79620.0,32480.286066,21791.999640,0.0,72838.0,32001.182637,21818.876333,Datasets/images/StateLineWeir_20140727_Farrell...


In [13]:
df2[["year", "month", "day"]] = df["SensorTime"].str.split("-", expand = True)

In [14]:
df2[["day", "time"]] = df2["day"].str.split(" ", expand = True)

In [15]:
df2

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma,images,year,month,day,time
0,2015-04-18 09:00:00,2015-04-18T08:57:44,StateLineWeir_20150418_Farrell_975.JPG,USGS,6674500,MDT,2.24,193.0,2020-03-11T23:03:07,4288,...,16362.030404,0.0,61859.0,29653.600833,16306.845477,Datasets/images/StateLineWeir_20150418_Farrell...,2012,06,09,13:15:00
1,2017-09-02 14:00:00,2017-09-02T13:59:49,StateLineWeir_20170902_Farrell_067.jpg,USGS,6674500,MDT,3.56,1630.0,2020-03-12T08:31:37,4288,...,28473.425686,0.0,118124.0,89406.285600,30296.151189,Datasets/images/StateLineWeir_20170902_Farrell...,2012,06,09,13:15:00
2,2018-04-07 17:15:00,2018-04-07T17:14:34,StateLineWeir_20180407_Farrell_339.jpg,USGS,6674500,MDT,2.29,224.0,2020-03-12T10:24:48,4288,...,18192.102384,0.0,64460.0,29188.783142,17965.580142,Datasets/images/StateLineWeir_20180407_Farrell...,2012,06,09,13:45:00
3,2017-06-06 11:00:00,2017-06-06T11:00:17,StateLineWeir_20170606_Farrell_642.jpg,USGS,6674500,MDT,4.58,3320.0,2020-03-12T07:08:01,4288,...,24985.793050,0.0,91560.0,41255.654072,24587.156055,Datasets/images/StateLineWeir_20170606_Farrell...,2012,06,09,14:45:00
4,2017-06-18 05:00:00,2017-06-18T05:00:15,StateLineWeir_20170618_Farrell_966.jpg,USGS,6674500,MDT,3.85,2080.0,2020-03-12T07:04:29,4288,...,20389.465149,0.0,69835.0,36189.672951,20127.291125,Datasets/images/StateLineWeir_20170618_Farrell...,2012,06,09,15:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42054,2017-04-11 05:15:00,2017-04-11T05:15:09,StateLineWeir_20170411_Farrell_070.jpg,USGS,6674500,MDT,2.79,579.0,2020-03-12T06:54:25,4288,...,20934.734921,0.0,76676.0,34905.093878,20211.447286,Datasets/images/StateLineWeir_20170411_Farrell...,2019,10,11,09:00:00
42055,2017-04-23 09:00:00,2017-04-23T09:00:12,StateLineWeir_20170423_Farrell_413.jpg,USGS,6674500,MDT,4.24,2720.0,2020-03-12T06:45:03,4288,...,0.000000,0.0,0.0,0.000000,0.000000,Datasets/images/StateLineWeir_20170423_Farrell...,2019,10,11,10:00:00
42056,2019-02-16 08:00:00,2019-02-16T07:59:20,StateLineWeir_20190216_Farrell_730.jpg,USGS,6674500,MST,2.17,164.0,2020-03-12T13:26:26,4288,...,10373.985930,0.0,33548.0,16201.727124,10127.155165,Datasets/images/StateLineWeir_20190216_Farrell...,2019,10,11,11:00:00
42057,2014-07-27 10:00:00,2014-07-27T10:00:18,StateLineWeir_20140727_Farrell_934.JPG,USGS,6674500,MDT,3.25,1470.0,2020-03-11T21:33:57,4288,...,21791.999640,0.0,72838.0,32001.182637,21818.876333,Datasets/images/StateLineWeir_20140727_Farrell...,2019,10,11,12:00:00


In [16]:
dfGroup = df2.groupby(df2['year'])

In [17]:
df2012 = dfGroup.get_group("2012")
df2013 = dfGroup.get_group("2013")
df2014 = dfGroup.get_group("2014")
df2015 = dfGroup.get_group("2015")
df2016 = dfGroup.get_group("2016")
df2017 = dfGroup.get_group("2017")
df2018 = dfGroup.get_group("2018")
df2019 = dfGroup.get_group("2019")


In [18]:
dfTrain = pd.concat([df2012, df2013, df2014, df2018, df2019])
dfTrain

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma,images,year,month,day,time
0,2015-04-18 09:00:00,2015-04-18T08:57:44,StateLineWeir_20150418_Farrell_975.JPG,USGS,6674500,MDT,2.24,193.0,2020-03-11T23:03:07,4288,...,16362.030404,0.0,61859.0,29653.600833,16306.845477,Datasets/images/StateLineWeir_20150418_Farrell...,2012,06,09,13:15:00
1,2017-09-02 14:00:00,2017-09-02T13:59:49,StateLineWeir_20170902_Farrell_067.jpg,USGS,6674500,MDT,3.56,1630.0,2020-03-12T08:31:37,4288,...,28473.425686,0.0,118124.0,89406.285600,30296.151189,Datasets/images/StateLineWeir_20170902_Farrell...,2012,06,09,13:15:00
2,2018-04-07 17:15:00,2018-04-07T17:14:34,StateLineWeir_20180407_Farrell_339.jpg,USGS,6674500,MDT,2.29,224.0,2020-03-12T10:24:48,4288,...,18192.102384,0.0,64460.0,29188.783142,17965.580142,Datasets/images/StateLineWeir_20180407_Farrell...,2012,06,09,13:45:00
3,2017-06-06 11:00:00,2017-06-06T11:00:17,StateLineWeir_20170606_Farrell_642.jpg,USGS,6674500,MDT,4.58,3320.0,2020-03-12T07:08:01,4288,...,24985.793050,0.0,91560.0,41255.654072,24587.156055,Datasets/images/StateLineWeir_20170606_Farrell...,2012,06,09,14:45:00
4,2017-06-18 05:00:00,2017-06-18T05:00:15,StateLineWeir_20170618_Farrell_966.jpg,USGS,6674500,MDT,3.85,2080.0,2020-03-12T07:04:29,4288,...,20389.465149,0.0,69835.0,36189.672951,20127.291125,Datasets/images/StateLineWeir_20170618_Farrell...,2012,06,09,15:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42054,2017-04-11 05:15:00,2017-04-11T05:15:09,StateLineWeir_20170411_Farrell_070.jpg,USGS,6674500,MDT,2.79,579.0,2020-03-12T06:54:25,4288,...,20934.734921,0.0,76676.0,34905.093878,20211.447286,Datasets/images/StateLineWeir_20170411_Farrell...,2019,10,11,09:00:00
42055,2017-04-23 09:00:00,2017-04-23T09:00:12,StateLineWeir_20170423_Farrell_413.jpg,USGS,6674500,MDT,4.24,2720.0,2020-03-12T06:45:03,4288,...,0.000000,0.0,0.0,0.000000,0.000000,Datasets/images/StateLineWeir_20170423_Farrell...,2019,10,11,10:00:00
42056,2019-02-16 08:00:00,2019-02-16T07:59:20,StateLineWeir_20190216_Farrell_730.jpg,USGS,6674500,MST,2.17,164.0,2020-03-12T13:26:26,4288,...,10373.985930,0.0,33548.0,16201.727124,10127.155165,Datasets/images/StateLineWeir_20190216_Farrell...,2019,10,11,11:00:00
42057,2014-07-27 10:00:00,2014-07-27T10:00:18,StateLineWeir_20140727_Farrell_934.JPG,USGS,6674500,MDT,3.25,1470.0,2020-03-11T21:33:57,4288,...,21791.999640,0.0,72838.0,32001.182637,21818.876333,Datasets/images/StateLineWeir_20140727_Farrell...,2019,10,11,12:00:00


In [19]:
dfTest = pd.concat([df2015, df2017])
dfTest

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma,images,year,month,day,time
10214,2016-10-17 17:00:00,2016-10-17T17:00:22,StateLineWeir_20161017_Farrell_220.jpg,USGS,6674500,MDT,2.57,408.0,2020-03-12T05:09:18,4288,...,19529.757589,0.0,68488.0,32461.898126,18836.324555,Datasets/images/StateLineWeir_20161017_Farrell...,2015,01,01,12:00:00
10215,2019-02-19 17:15:00,2019-02-19T17:14:18,StateLineWeir_20190219_Farrell_829.jpg,USGS,6674500,MST,2.15,159.0,2020-03-12T13:24:49,4288,...,14355.818500,0.0,55413.0,22743.389886,13914.602485,Datasets/images/StateLineWeir_20190219_Farrell...,2015,01,01,13:00:00
10216,2019-03-26 05:30:00,2019-03-26T05:29:11,StateLineWeir_20190326_Farrell_860.jpg,USGS,6674500,MDT,2.26,214.0,2020-03-12T13:46:22,4288,...,11461.751954,0.0,40234.0,18634.472951,10996.006519,Datasets/images/StateLineWeir_20190326_Farrell...,2015,01,01,14:00:00
10217,2018-04-11 05:45:00,2018-04-11T05:44:35,StateLineWeir_20180411_Farrell_453.jpg,USGS,6674500,MDT,2.37,270.0,2020-03-12T10:22:04,4288,...,19331.849866,0.0,71174.0,34166.992623,18893.716675,Datasets/images/StateLineWeir_20180411_Farrell...,2015,01,01,15:00:00
10218,2018-06-09 07:00:00,2018-06-09T06:59:39,StateLineWeir_20180609_Farrell_216.jpg,USGS,6674500,MDT,2.37,281.0,2020-03-12T11:38:37,4288,...,18529.841605,0.0,78948.0,24242.978654,17076.918128,Datasets/images/StateLineWeir_20180609_Farrell...,2015,01,01,16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28806,2015-05-07 07:00:00,2015-05-07T06:57:43,StateLineWeir_20150507_Farrell_403.JPG,USGS,6674500,MDT,2.28,231.0,2020-03-11T23:44:38,4288,...,16873.629265,0.0,59977.0,33747.380719,16925.568757,Datasets/images/StateLineWeir_20150507_Farrell...,2017,12,31,13:00:00
28807,2016-08-04 10:00:00,2016-08-04T10:00:24,StateLineWeir_20160804_Farrell_099.jpg,USGS,6674500,MDT,3.58,1830.0,2020-03-12T04:44:18,4288,...,26614.128966,0.0,89150.0,43352.536526,26254.586103,Datasets/images/StateLineWeir_20160804_Farrell...,2017,12,31,14:00:00
28808,2016-03-27 17:00:00,2016-03-27T17:00:24,StateLineWeir_20160327_Farrell_041.JPG,USGS,6674500,MDT,2.29,243.0,2020-03-12T02:22:48,4288,...,15666.490763,0.0,63557.0,30321.158537,15079.066090,Datasets/images/StateLineWeir_20160327_Farrell...,2017,12,31,15:00:00
28809,2012-12-04 09:45:00,2012-12-04T09:51:59,StateLineWeir_20121204_Farrell_041.JPG,USGS,6674500,MST,2.27,214.0,2020-03-11T19:01:08,4288,...,0.000000,0.0,0.0,0.000000,0.000000,Datasets/images/StateLineWeir_20121204_Farrell...,2017,12,31,16:00:00


In [20]:
dfVal = df2016
dfVal

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma,images,year,month,day,time
15888,2018-11-08 07:00:00,2018-11-08T06:59:34,StateLineWeir_20181108_Farrell_754.jpg,USGS,6674500,MST,2.32,244.0,2020-03-12T12:41:06,4288,...,16203.098104,0.0,55285.0,26813.250820,16027.270749,Datasets/images/StateLineWeir_20181108_Farrell...,2016,01,01,10:00:00
15889,2018-03-22 11:00:00,2018-03-22T10:59:32,StateLineWeir_20180322_Farrell_852.jpg,USGS,6674500,MDT,2.25,202.0,2020-03-12T09:51:17,4288,...,17904.544391,0.0,70412.0,36289.296965,18064.169456,Datasets/images/StateLineWeir_20180322_Farrell...,2016,01,01,11:00:00
15890,2014-07-03 13:00:00,2014-07-03T13:00:20,StateLineWeir_20140703_Farrell_376.JPG,USGS,6674500,MDT,3.17,1270.0,2020-03-11T21:44:29,4288,...,22956.455812,0.0,83077.0,36056.046760,22720.787956,Datasets/images/StateLineWeir_20140703_Farrell...,2016,01,01,12:00:00
15891,2019-01-30 09:00:00,2019-01-30T08:59:27,StateLineWeir_20190130_Farrell_228.jpg,USGS,6674500,MST,2.26,0.0,2020-03-12T13:40:25,4288,...,16740.389400,0.0,73833.0,30367.067323,15532.107586,Datasets/images/StateLineWeir_20190130_Farrell...,2016,01,01,13:00:00
15892,2012-06-24 12:00:00,2012-06-24T12:02:05,StateLineWeir_20120624_Farrell_339.jpg,USGS,6674500,MDT,3.25,1310.0,2020-03-11T17:11:51,4288,...,23917.253594,0.0,85689.0,44145.484824,23576.994053,Datasets/images/StateLineWeir_20120624_Farrell...,2016,01,01,14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21416,2014-08-11 11:00:00,2014-08-11T11:00:19,StateLineWeir_20140811_Farrell_286.JPG,USGS,6674500,MDT,3.22,1420.0,2020-03-11T22:02:41,4288,...,26002.825186,0.0,91525.0,34683.137759,25869.204119,Datasets/images/StateLineWeir_20140811_Farrell...,2016,12,31,13:00:00
21417,2014-09-18 06:30:00,2014-09-18T06:30:20,StateLineWeir_20140918_Farrell_110.JPG,USGS,6674500,MDT,3.49,1700.0,2020-03-11T22:21:41,4288,...,26740.657830,0.0,78195.0,38732.721817,26825.178050,Datasets/images/StateLineWeir_20140918_Farrell...,2016,12,31,14:00:00
21418,2018-05-17 17:45:00,2018-05-17T17:44:41,StateLineWeir_20180517_Farrell_544.jpg,USGS,6674500,MDT,2.33,235.0,2020-03-12T10:41:30,4288,...,17960.262129,0.0,61662.0,28385.331974,17663.272638,Datasets/images/StateLineWeir_20180517_Farrell...,2016,12,31,15:00:00
21419,2019-04-11 05:30:00,2019-04-11T05:29:14,StateLineWeir_20190411_Farrell_335.jpg,USGS,6674500,MDT,2.22,193.0,2020-03-12T14:20:25,4288,...,9243.868341,0.0,32512.0,18207.252666,8425.124433,Datasets/images/StateLineWeir_20190411_Farrell...,2016,12,31,16:00:00


In [21]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu',input_shape=(512,512,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='linear'))

In [22]:
model.compile(optimizer='adam', metrics=['mse'], loss=[tf.keras.metrics.MeanSquaredError()])

In [27]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_dataframe(
    dfTrain,
    directory='Datasets/images/',
    x_col='images',
    y_col=' Discharge',
    target_size=(512,512),
    batch_size=8,
    class_mode='raw',
    validate_filenames=True
)

val_data = datagen.flow_from_dataframe(
    dfVal,
    directory='Datasets/images/',
    x_col='images',
    y_col=' Discharge',
    target_size=(512,512),
    batch_size=8,
    class_mode='raw',
    validate_filenames=True
)


Found 0 validated image filenames.
Found 0 validated image filenames.


In [26]:
history = model.fit(train_data, epochs=5, validation_data=val_data)

ValueError: Asked to retrieve element 0, but the Sequence has length 0